In [272]:
import pandas as pd

# Set pandas options to display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping of columns

df = pd.read_excel("kolkata_cars.xlsx")
length = len(df)
df.columns

Index(['new_car_detail', 'new_car_overview', 'new_car_feature',
       'new_car_specs', 'car_links'],
      dtype='object')

New_Car_Details

In [273]:
import ast

data_list = df['new_car_detail'].to_list()

# Function to convert string to dictionary
def to_dict(data):
    if isinstance(data,str):
        try:
            return ast.literal_eval(data)
        except(ValueError,SyntaxError):
            return {}
    return data

data_list = [to_dict(item) for item in data_list]

new_car_detail_flatened_data = pd.DataFrame(data_list)
new_car_detail_flatened_data['row index'] = range(length)

# # Function to flatten a single dictionary
# def flatten_dict(data):
#     if not isinstance(data, dict):
#         return {}
    
#     if 'trendingText' in data:
#         # Flatten the nested dictionary
#         flat_data = {
#             **data,  # Unpack the main dictionary
#             **{f'trendingText_{k}': v for k, v in data['trendingText'].items()}  # Flatten the nested dictionary
#         }
#         del flat_data['trendingText']  # Remove the original nested dictionary
#     else:
#         # If 'trendingText' key is missing, just return the data as is
#         flat_data = data
#     return flat_data
# flattened_data_list = [flatten_dict(row) for row in data_list]
new_car_detail_flatened_data.head()

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText,row index
0,0,Petrol,Sedan,"70,000",Automatic,3,3rd Owner,Toyota,Toyota Camry,2014,2123,Hybrid,₹ 9.75 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,0
1,0,Petrol,Hatchback,"23,981",Manual,1,1st Owner,Datsun,Datsun RediGO,2017,5236,T Option,₹ 2.66 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,1
2,0,Petrol,SUV,"7,100",Automatic,1,1st Owner,Renault,Renault Kiger,2021,8352,RXT AMT,₹ 5.95 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,2
3,0,Petrol,Hatchback,"71,574",Manual,2,2nd Owner,Hyundai,Hyundai i20,2011,1642,1.2 Sportz,₹ 2.23 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,3
4,0,Diesel,SUV,"50,000",Automatic,2,2nd Owner,Audi,Audi Q3,2014,99,35 TDI Quattro Premium,₹ 12 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,4


New_Car_Overview

In [274]:
import pandas as pd
import ast

# Convert each string to a dictionary
nested_dict_list = [ast.literal_eval(d) for d in df['new_car_overview']]

# Function to extract and flatten each dictionary
def extract_flatten_dict(nested_dict):
    if 'top' in nested_dict:
        top_data = nested_dict['top']
        df = pd.DataFrame(top_data)
        df = df.drop(columns=['icon'], errors='ignore')  # Drop the 'icon' column if present
        return df.set_index('key').T
    else:
        return pd.DataFrame()

# Extract and flatten all dictionaries
df_list = [extract_flatten_dict(d) for d in nested_dict_list]

# Concatenate all DataFrames into one
new_car_overview_df_combined = pd.concat(df_list, ignore_index=True)

# Display the combined DataFrame
new_car_overview_df_combined['row index'] = range(length)
new_car_overview_df_combined.head()

key,Registration Year,Insurance Validity,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture,row index
0,2014,Third Party insurance,Petrol,5 Seats,"70,000 Kms",WB02,Third Owner,2494 cc,Automatic,2014,0
1,Jul 2017,Comprehensive,Petrol,5 Seats,"23,981 Kms",WB24,First Owner,799 cc,Manual,2017,1
2,Nov 2021,Comprehensive,Petrol,5 Seats,"7,100 Kms",WB06,First Owner,999 cc,Automatic,2021,2
3,Feb 2012,Third Party,Petrol,5 Seats,"71,574 Kms",WB06,Second Owner,1197 cc,Manual,2011,3
4,2014,Third Party insurance,Diesel,5 Seats,"50,000 Kms",WB02,Second Owner,1968 cc,Automatic,2014,4


Car_Feature

In [275]:
import pandas as pd
import ast

def extract_features(data_dict):
    # Extract top features
    top_features = [item['value'] for item in data_dict.get('top', [])]
    
    # Extract detailed features
    detailed_features = []
    for category in data_dict.get('data', []):
        heading = category.get('heading', '')
        subHeading = category.get('subHeading', '')
        for item in category.get('list', []):
            detailed_features.append({
                'Category': heading,
                'SubCategory': subHeading,
                'Feature': item['value']
            })
    
    # Convert to DataFrames
    df_top = pd.DataFrame(top_features, columns=['Top Feature'])
    df_detailed = pd.DataFrame(detailed_features)
    
    return df_top, df_detailed

df_ = pd.DataFrame(df, columns=['new_car_feature'])

# Initialize lists to store results
top_features_list = []
detailed_features_list = []

# Process each row
for index, row in df_.iterrows():
    # Convert the string to a dictionary
    try:
        data_dict = ast.literal_eval(row['new_car_feature'])
    except (ValueError, SyntaxError):
        print(f"Error parsing row {index}")
        continue
    
    # Extract features
    df_top_features, df_detailed_features = extract_features(data_dict)
    
    # Add row index for reference
    df_top_features['Row Index'] = index
    df_detailed_features['Row Index'] = index
    
    # Append to lists
    top_features_list.append(df_top_features)
    detailed_features_list.append(df_detailed_features)

# Combine all DataFrames
df_top_combined = pd.concat(top_features_list, ignore_index=True)
df_detailed_combined = pd.concat(detailed_features_list, ignore_index=True)

df_top_pivot = df_top_combined.groupby('Row Index')['Top Feature'].apply(lambda x: ', '.join(x)).reset_index()

# Rename columns
df_top_pivot.columns = ['row index', 'Top Features']

# Display the transformed DataFrame
df_top_pivot.head()


,row index,Top Features
0,0,"Power Steering, Power Windows Front, Air Condi..."
1,1,"Power Steering, Power Windows Front, Air Condi..."
2,2,"Power Steering, Power Windows Front, Air Condi..."
3,3,"Power Steering, Power Windows Front, Air Condi..."
4,4,"Power Steering, Power Windows Front, Air Condi..."


New_Car_Specs

In [276]:
import pandas as pd
import ast

car_spec_list = df['new_car_specs'].to_list()

# Define a function to process each row and extract key-value pairs
def extract_key_values(data):
    # Convert string to dictionary
    nested_dict = ast.literal_eval(data)
    
    # Extract key-value pairs from the 'data' section
    data_list = nested_dict.get('data', [])
    key_value_dict = {}
    for item in data_list:
        for kv in item.get('list', []):
            key_value_dict[kv['key']] = kv['value']
    return key_value_dict

# Sample DataFrame with 'new_car_specs' column
df = pd.DataFrame({
    'new_car_specs': car_spec_list
})

# Apply the extraction function to each row
df['key_value_dict'] = df['new_car_specs'].apply(extract_key_values)

# Convert the 'key_value_dict' column into a DataFrame
key_value_df = pd.json_normalize(df['key_value_dict'])
key_value_df['row index'] = range(length)
# Display the DataFrame
key_value_df


,Color,Engine Type,Displacement,Max Power,Max Torque,No of Cylinder,Values per Cylinder,Value Configuration,Fuel Suppy System,BoreX Stroke,Turbo Charger,Super Charger,Length,Width,Height,Wheel Base,Front Tread,Rear Tread,Kerb Weight,Gross Weight,Gear Box,Drive Type,Seating Capacity,Steering Type,Turning Radius,Front Brake Type,Rear Brake Type,Tyre Type,Alloy Wheel Size,No Door Numbers,Ground Clearance Unladen,Cargo Volumn,Compression Ratio,Acceleration,Top Speed,row index
0,White,2AR FXE Petrol Engine,2494,158.2bhp@5700rpm,213Nm@4500rpm,4.0,4.0,DOHC,Electronic Fuel Injection(EFI),90 X 98 mm,No,No,4825mm,1825mm,1480mm,2775mm,1580mm,1570mm,1625kg,2100kg,6 Speed,FWD,5,Power,5.5 metres,Ventilated Disc,Solid Disc,"Tubeless,Radial",17,4,NaN,NaN,NaN,NaN,NaN,0
1,Red,0.8L Petrol Engine,799,53.64bhp@5678rpm,72Nm@4386rpm,3.0,4.0,DOHC,MPFi,NaN,No,No,3429mm,1560mm,1541mm,2348mm,NaN,NaN,770,NaN,5-Speed,2WD,5,Power,4.7m,Disc,Drum,NaN,NaN,5,185mm,222,NaN,NaN,NaN,1
2,Blue,1.0L energy,999,71.01bhp@6250rpm,96Nm@3500rpm,3.0,4.0,NaN,MPFi,NaN,NaN,NaN,3991mm,1750,1605,2500,1536,1535,1055,NaN,5 Speed,NaN,5,Electric,NaN,Disc,Drum,"Tubeless, Radial",NaN,5,NaN,405,NaN,NaN,NaN,2
3,Red,In-Line Engine,1197,80 PS at 5200 rpm,"11.4 kgm at 4,000 rpm",4.0,4.0,DOHC,MPFI,NaN,No,No,3940 mm,1710 mm,1505 mm,"2,525 mm","1,505 mm","1,503 mm",1020,NaN,5 Speed,FWD,5,Power,5.20 m,Disc,Drum,"Tubeless,Radial",14,5,NaN,295 Lit,:1,12.96 Sec,NaN,3
4,White,TDI Diesel Engine,1968,174.33bhp@4200rpm,380Nm@1750-2500rpm,4.0,4.0,DOHC,CRDI,NaN,Yes,No,4385mm,2019mm,1608mm,2603mm,1571mm,1575mm,1660kg,2185kg,7-Speed S-Tronic,AWD,5,Power,5.9 metres,Ventilated Disc,Drum,"Tubeless,Radial",16,5,NaN,460-litres,NaN,8.2 Seconds,212 Kmph,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,Others,K10C,998,65.71bhp@5500rpm,89Nm@3500rpm,3.0,4.0,NaN,NaN,NaN,NaN,NaN,3695mm,1655,1555,2435,1430,1440,810,1260,5-Speed,NaN,5,Electric,NaN,Ventilated Disc,Drum,"Tubeless, Radial",NaN,5,NaN,313,NaN,NaN,NaN,1376
1377,Others,F8D Petrol Engine,796,47.3bhp@6000rpm,69Nm@3500rpm,3.0,4.0,DOHC,MPFi,NaN,No,No,3395mm,1490mm,1475mm,2360mm,1295mm,1290mm,720kg,1185kg,5 Speed,FWD,5,Power,4.6 metres,Solid Disc,Drum,Tubeless Tyres,NaN,5,NaN,177-litres,NaN,19 Seconds,140 kmph,1377
1378,Others,Blue Efficiency Engine,1796,186bhp@5600rpm,285Nm@2400-4000rpm,4.0,4.0,NaN,Mpfi,NaN,No,No,4591mm,1770mm,1447mm,2760mm,1549mm,1552mm,1585kg,2020kg,7 Speed,RWD,5,Power,5.42 metres,Ventilated Disc,Solid Disc,"Tubeless,Radial",17,4,NaN,475-litres,9.3:1,8.8 Seconds,230km/hr,1378
1379,Others,K Series Petrol Engine,1197,85.80bhp@6000rpm,114Nm@4000rpm,4.0,4.0,DOHC,MPFI,NaN,No,No,3775mm,1680mm,1620mm,2360mm,1470mm,1480mm,1030kg,1430kg,5 Speed,FWD,5,Power,4.7 metres,Ventilated Disc,Drum,"Tubeless,Radial",14,5,NaN,236-liters,NaN,15 Seconds,156 Kmph,1379


In [277]:
import pandas as pd

# Assuming the dataframes are loaded and named accordingly
# new_car_detail_flatened_data
# new_car_overview_df_combined
# df_top_pivot
# key_value_df

# First, merge new_car_detail_flatened_data with new_car_overview_df_combined
merged_df = pd.merge(new_car_detail_flatened_data, new_car_overview_df_combined, on='row index', how='outer')

# Then, merge the result with df_top_pivot
merged_df = pd.merge(merged_df, df_top_pivot, on='row index', how='outer')

# Finally, merge with key_value_df
merged_df = pd.merge(merged_df, key_value_df, on='row index', how='outer')

# Display the final combined DataFrame
merged_df.head()


,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText,row index,Registration Year,Insurance Validity,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture,Top Features,Color,Engine Type,Displacement,Max Power,Max Torque,No of Cylinder,Values per Cylinder,Value Configuration,Fuel Suppy System,BoreX Stroke,Turbo Charger,Super Charger,Length,Width,Height,Wheel Base,Front Tread,Rear Tread,Kerb Weight,Gross Weight,Gear Box,Drive Type,Seating Capacity,Steering Type,Turning Radius,Front Brake Type,Rear Brake Type,Tyre Type,Alloy Wheel Size,No Door Numbers,Ground Clearance Unladen,Cargo Volumn,Compression Ratio,Acceleration,Top Speed
0,0,Petrol,Sedan,"70,000",Automatic,3,3rd Owner,Toyota,Toyota Camry,2014,2123,Hybrid,₹ 9.75 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,0,2014,Third Party insurance,Petrol,5 Seats,"70,000 Kms",WB02,Third Owner,2494 cc,Automatic,2014,"Power Steering, Power Windows Front, Air Condi...",White,2AR FXE Petrol Engine,2494,158.2bhp@5700rpm,213Nm@4500rpm,4.0,4.0,DOHC,Electronic Fuel Injection(EFI),90 X 98 mm,No,No,4825mm,1825mm,1480mm,2775mm,1580mm,1570mm,1625kg,2100kg,6 Speed,FWD,5,Power,5.5 metres,Ventilated Disc,Solid Disc,"Tubeless,Radial",17,4,NaN,NaN,NaN,NaN,NaN
1,0,Petrol,Hatchback,"23,981",Manual,1,1st Owner,Datsun,Datsun RediGO,2017,5236,T Option,₹ 2.66 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,1,Jul 2017,Comprehensive,Petrol,5 Seats,"23,981 Kms",WB24,First Owner,799 cc,Manual,2017,"Power Steering, Power Windows Front, Air Condi...",Red,0.8L Petrol Engine,799,53.64bhp@5678rpm,72Nm@4386rpm,3.0,4.0,DOHC,MPFi,NaN,No,No,3429mm,1560mm,1541mm,2348mm,NaN,NaN,770,NaN,5-Speed,2WD,5,Power,4.7m,Disc,Drum,NaN,NaN,5,185mm,222,NaN,NaN,NaN
2,0,Petrol,SUV,"7,100",Automatic,1,1st Owner,Renault,Renault Kiger,2021,8352,RXT AMT,₹ 5.95 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,2,Nov 2021,Comprehensive,Petrol,5 Seats,"7,100 Kms",WB06,First Owner,999 cc,Automatic,2021,"Power Steering, Power Windows Front, Air Condi...",Blue,1.0L energy,999,71.01bhp@6250rpm,96Nm@3500rpm,3.0,4.0,NaN,MPFi,NaN,NaN,NaN,3991mm,1750,1605,2500,1536,1535,1055,NaN,5 Speed,NaN,5,Electric,NaN,Disc,Drum,"Tubeless, Radial",NaN,5,NaN,405,NaN,NaN,NaN
3,0,Petrol,Hatchback,"71,574",Manual,2,2nd Owner,Hyundai,Hyundai i20,2011,1642,1.2 Sportz,₹ 2.23 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,3,Feb 2012,Third Party,Petrol,5 Seats,"71,574 Kms",WB06,Second Owner,1197 cc,Manual,2011,"Power Steering, Power Windows Front, Air Condi...",Red,In-Line Engine,1197,80 PS at 5200 rpm,"11.4 kgm at 4,000 rpm",4.0,4.0,DOHC,MPFI,NaN,No,No,3940 mm,1710 mm,1505 mm,"2,525 mm","1,505 mm","1,503 mm",1020,NaN,5 Speed,FWD,5,Power,5.20 m,Disc,Drum,"Tubeless,Radial",14,5,NaN,295 Lit,:1,12.96 Sec,NaN
4,0,Diesel,SUV,"50,000",Automatic,2,2nd Owner,Audi,Audi Q3,2014,99,35 TDI Quattro Premium,₹ 12 Lakh,,,None,{'imgUrl': 'https://stimg.cardekho.com/used-ca...,4,2014,Third Party insurance,Diesel,5 Seats,"50,000 Kms",WB02,Second Owner,1968 cc,Automatic,2014,"Power Steering, Power Windows Front, Air Condi...",White,TDI Diesel Engine,1968,174.33bhp@4200rpm,380Nm@1750-2500rpm,4.0,4.0,DOHC,CRDI,NaN,Yes,No,4385mm,2019mm,1608mm,2603mm,1571mm,1575mm,1660kg,2185kg,7-Speed S-Tronic,AWD,5,Power,5.9 metres,Ventilated Disc,Drum,"Tubeless,Radial",16,5,NaN,460-litres,NaN,8.2 Seconds,212 Kmph


In [278]:
merged_df.columns = merged_df.columns.str.lower()
merged_df.columns = merged_df.columns.str.replace(" ","_")

In [279]:
merged_df.to_csv('kolkata.csv')

In [280]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1381 entries, 0 to 1380
Data columns (total 64 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   it                        1381 non-null   int64  
 1   ft                        1381 non-null   object 
 2   bt                        1381 non-null   object 
 3   km                        1381 non-null   object 
 4   transmission              1381 non-null   object 
 5   ownerno                   1381 non-null   int64  
 6   owner                     1381 non-null   object 
 7   oem                       1381 non-null   object 
 8   model                     1381 non-null   object 
 9   modelyear                 1381 non-null   int64  
 10  centralvariantid          1381 non-null   int64  
 11  variantname               1381 non-null   object 
 12  price                     1381 non-null   object 
 13  priceactual               1381 non-null   object 
 14  pricesav